## Fitting the Model to the Scotland Lip Cancer Dataset

In this section, we'll implement the Besag-York-Mollié (BYM) model for the Scotland dataset using [CmdStanPy](https://mc-stan.org/cmdstanpy/), a lightweight interface to Stan for Python. We'll use the Python standard library's [json](https://docs.python.org/3/library/json.html) module to load and parse our data, and [numpy](https://numpy.org/) for numerical operations.

In [ ]:
import cmdstanpy
from cmdstanpy import CmdStanModel
import numpy as np
import json

In [ ]:
if not cmdstanpy.cmdstan_path():
    cmdstanpy.install_cmdstan()

with open("data/scotland_data.json") as file:
    scotland_data = json.load(file)

def munge_car_data_for_stan(adjBUGS, numBUGS):
    N = len(numBUGS)
    nn = numBUGS
    N_edges = len(adjBUGS) // 2
    node1 = np.zeros(N_edges, dtype=int)
    node2 = np.zeros(N_edges, dtype=int)
    iAdj = 0
    iEdge = 0
    
    for i in range(N):
        for j in range(nn[i]):
            iAdj += 1
            if i + 1 < adjBUGS[iAdj - 1]:
                iEdge += 1
                node1[iEdge - 1] = i + 1
                node2[iEdge - 1] = adjBUGS[iAdj - 1]
    
    return {
        "N": N,
        "N_edges": N_edges,
        "node1": node1,
        "node2": node2
    }

In [ ]:
nbs = munge_car_data_for_stan(
    scotland_data["adj"], scotland_data["num"]
)

scotland_data_subset = {
    "y": np.array(scotland_data["y"]),
    "x": np.array(scotland_data["x"]) * 0.1,
    "E": np.array(scotland_data["E"])
}

data = {**nbs, **scotland_data_subset}
bym_model = CmdStanModel(stan_file="stan/bym_predictor_plus_offset.stan")
bym_scot_stanfit = bym_model.sample(data=data, show_progress=True)

In [ ]:
vars = [
    "lp__",
    "beta0",
    "beta1",
    "sigma_phi",
    "tau_phi",
    "sigma_theta",
    "tau_theta",
    "mu[5]",
    "phi[5]",
    "theta[5]",
]
bym_scot_stanfit.summary().loc[vars]

## BYM2: improving the parameterization of the Besag, York, and Mollié model

In the following section, we implement the BYM2 model, which improves upon the parameterization of the original BYM model. As there is no native implementation of INLA in Python, we calculate the scaling factor from scratch using the [SciPy](https://scipy.org/) library.

In [ ]:
import scipy.sparse as sp

def get_scaling_factor(node1, node2, N):
     # Convert to 0-indexing
    node1 = node1 - 1
    node2 = node2 - 1

    # Create adjacency matrix based on node1 and node2
    A = sp.coo_matrix((np.ones(len(node1)), (node1, node2)), shape=(N, N))

    # Create the ICAR precision matrix
    A = A + A.T
    Q = sp.diags(A.sum(axis=1).A1) - A

    # Compute the pseudo-inverse of the precision matrix
    Q_pinv = np.linalg.pinv(Q.todense())
        
    # Extract the diagonal elements
    cov_diag = np.diag(Q_pinv)

    # Compute the geometric mean of the variances
    scaling_factor = np.exp(np.mean(np.log(cov_diag)))
    
    return scaling_factor

In [ ]:
nbs = munge_car_data_for_stan(
    scotland_data["adj"], scotland_data["num"]
)

scotland_data_subset = {
    "y": np.array(scotland_data["y"]),
    "x": np.array(scotland_data["x"]) * 0.1,
    "E": np.array(scotland_data["E"])
}

scaling_factor = get_scaling_factor(nbs["node1"], nbs["node2"], nbs["N"])

data = {**nbs, **scotland_data_subset, **{"scaling_factor": scaling_factor}}
bym2_model = CmdStanModel(stan_file="stan/bym2_predictor_plus_offset.stan")
bym2_scot_stanfit = bym2_model.sample(data=data, show_progress=True)

In [ ]:
vars = [
    "lp__",
    "beta0",
    "beta1",
    "sigma",  "rho", "mu[5]",
    "phi[5]", "theta[5]"
]
bym2_scot_stanfit.summary().loc[vars]

## Bigger data: from 56 counties in Scotland to 1921 census tracts in New York City

In this example, we'll apply the BYM2 model to a dataset of reported traffic accidents in New York City involving cars and either pedestrians or bicyclists. The data, collected between 2001 and 2009, is aggregated at the census tract level. For this analysis, we'll focus on the 2001 data, which covers 1,921 census tracts.
We begin by loading the dataset, `nyc_subset.json`, using [Pandas](https://pandas.pydata.org/). This file contains a list of the 1,921 census tract IDs (`nyc_tractIDs`), the count of injuries per tract in 2001 (`events_2001`), and the 2001 population per census tract (`pop_2001`). To visualize the relationship between injury counts and population across census tracts, we'll create a scatterplot using [plotnine](https://plotnine.org/), a Python implementation of the grammar of graphics.

In [ ]:
import pandas as pd
import plotnine as p9

p9.theme_set(
    p9.theme_grey()
    + p9.theme(
        text=p9.element_text(size=10),
        plot_title=p9.element_text(size=14),
        axis_title_x=p9.element_text(size=12),
        axis_title_y=p9.element_text(size=12),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8),
    )
)

nyc_subset = pd.read_json("data/nyc_subset.json", dtype={"nyc_tractIDs": str})

(
    p9.ggplot(data=nyc_subset, mapping=p9.aes(x="pop_2001", y="events_2001")) + 
    p9.geom_point() + 
    p9.scale_x_continuous(trans="log")
)

To incorporate spatial information for New York City, we'll use data stored in the `nycTracts10` directory. We'll load this spatial data using [GeoPandas](https://geopandas.org/en/stable/), a powerful library for handling geospatial data in Python. In addition to GeoPandas, we'll utilize [libpysal](https://pysal.org/libpysal/), a Python spatial analysis library to generate a list of neighboring census tracts for each tract.

In [ ]:
import geopandas as gpd

nyc_shp = gpd.read_file("data/nycTracts10/nycTracts10.shp")
nyc_tractIDs = nyc_subset["nyc_tractIDs"]
nyc_shp_subset = nyc_shp.merge(
    nyc_subset, how="inner", left_on="GEOID10", right_on="nyc_tractIDs"
).assign(log_pop_2001=lambda df: np.log(df["pop_2001"] + 0.1))

In [ ]:
import libpysal as sa

# Create a spatial weights object
nyc_nbs = sa.weights.Queen(nyc_shp_subset['geometry'])

# List of nodes
nodes = nyc_nbs.neighbors.keys()

# List of edges
edges = [(node+1, neighbor+1) for node in nodes for neighbor in nyc_nbs.neighbors[node]]

# Unzip edges to get node1 and node2 and convert to numpy arrays
node1, node2 = zip(*edges)
node1 = np.array(node1)
node2 = np.array(node2)

# Compute the scaling factor
scaling_factor = get_scaling_factor(node1, node2, len(nyc_shp_subset))

N = len(nyc_shp_subset)
y = nyc_shp_subset["events_2001"].values
E = nyc_shp_subset["pop_2001"].values

# Set population > 0
E[E < 10] = 10

data = {
    "N": N,
    "N_edges": len(edges),
    "node1": node1,
    "node2": node2,
    "y": y,
    "E": E,
    "scaling_factor": scaling_factor
}

bym2_model = CmdStanModel(stan_file="stan/bym2_offset_only.stan")
bym2_nyc_stanfit = bym2_model.sample(data=data, show_progress=True)

In [ ]:
vars = [
    "beta0", "rho", "sigma","mu[1]", "mu[2]", "mu[3]", "mu[500]", 
    "mu[1000]", "mu[1500]", "mu[1900]", "phi[1]", "phi[2]", "phi[3]", 
    "phi[500]", "phi[1000]", "phi[1500]", "phi[1900]", "theta[1]", 
    "theta[2]", "theta[3]", "theta[500]", "theta[1000]", "theta[1500]", 
    "theta[1900]"
]

bym2_nyc_model_summary = bym2_nyc_stanfit.summary()
bym2_nyc_model_summary.loc[vars]

To visualize the spatial data, model fits, and neighborhood graph on an interactive map, we utilize [Folium](https://python-visualization.github.io/folium/latest/). This Python library is particularly useful for creating leaflet.js maps, allowing us to display the geographical boundaries of the census tracts and create a choropleth map based on injury counts and population.

In [ ]:
import folium

centroids = nyc_shp_subset['geometry'].centroid
m = folium.Map(location=[centroids.y.mean(), centroids.x.mean()], zoom_start=12)

# Add the geometries to the folium map
for geom in nyc_shp_subset.geometry:
    geo_json = folium.GeoJson(data=geom.__geo_interface__)
    geo_json.add_to(m)

for i, neighbors in nyc_nbs.neighbors.items():
    for neighbor in neighbors:
        # Get the coordinates of the centroids of neighboring polygons
        p1 = [centroids.iloc[i].y, centroids.iloc[i].x]
        p2 = [centroids.iloc[neighbor].y, centroids.iloc[neighbor].x]

        # Add a line between the centroids of neighboring polygons
        folium.PolyLine([p1, p2], color="blue", weight=2).add_to(m)

m

In the following plot, the first panel shows the 2001 log population per census tract and the second panel shows the raw number of 2001 events.

In [ ]:
# Create the map centered on the centroid of the GeoDataFrame
m = folium.Map(location=[40.71380198, -73.91687195], zoom_start=10)

# Add the choropleth layer
folium.Choropleth(
    geo_data=nyc_shp_subset.to_json(),
    data=nyc_shp_subset,
    columns=['nyc_tractIDs', 'log_pop_2001'],
    key_on='feature.properties.nyc_tractIDs',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Log population per tract in 2001'
).add_to(m)

m

In [ ]:
# Create the map centered on the centroid of the GeoDataFrame
m = folium.Map(location=[40.71380198, -73.91687195], zoom_start=10.5)

# Add the choropleth layer
folium.Choropleth(
    geo_data=nyc_shp_subset.to_json(),
    data=nyc_shp_subset.assign(events_2001 = lambda df: np.clip(df['events_2001'], 0, 15)),
    columns=['nyc_tractIDs', 'events_2001'],
    key_on='feature.properties.nyc_tractIDs',
    fill_color='Blues',
    line_opacity=0.1,
    legend_name='Raw number of events per tract in 2001'
).add_to(m)

m

#### Baseline model: a simple Poisson GLM

In [ ]:
N = len(nyc_shp_subset["events_2001"])
y = nyc_shp_subset["events_2001"]
E = nyc_shp_subset["pop_2001"]

# Set population > 0
E[E < 10] = 10

data = {"N": N, "y": y, "E": E}
pois_model = CmdStanModel(stan_file="stan/pois.stan")
pois_model_stanfit = pois_model.sample(data=data)

In [ ]:
vars = [
    "beta0",
    "mu[1]",
    "mu[2]",
    "mu[3]",
    "mu[500]",
    "mu[1000]",
    "mu[1500]",
    "mu[1900]",
]

pois_model_summary = pois_model_stanfit.summary()
pois_model_summary.loc[vars]

In [ ]:
def plot_map(df, var):
    m = folium.Map(location=[40.71380198, -73.91687195], zoom_start=10.5)
    folium.Choropleth(
        geo_data=df.to_json(),
        data=df,
        columns=["nyc_tractIDs", var],
        key_on="feature.properties.nyc_tractIDs",
        fill_color="Blues",
        line_opacity=0.1,
    ).add_to(m)
    return m

In [ ]:
mu_index = pois_model_summary.index.str.contains("mu")
mu_mean = pois_model_summary.loc[mu_index, "Mean"].reset_index(drop=True)
plot_map(nyc_shp_subset.assign(mu = mu_mean), "mu")

#### Adding a vector of random effects (heterogeneous variation only)

In [ ]:
N = len(nyc_shp_subset["events_2001"])
y = nyc_shp_subset["events_2001"]
E = nyc_shp_subset["pop_2001"]

# Set population > 0
E[E < 10] = 10

data = {"N": N, "y": y, "E": E}

pois_re_model = CmdStanModel(stan_file="stan/pois_re.stan")
pois_re_model_stanfit = pois_re_model.sample(data=data)

In [ ]:
vars = [
    "beta0",
    "mu[1]",
    "mu[2]",
    "mu[3]",
    "mu[500]",
    "mu[1000]",
    "mu[1500]",
    "mu[1900]",
    "theta[1]", 
    "theta[2]", "theta[3]", 
    "theta[500]", 
    "theta[1000]", 
    "theta[1500]", 
    "theta[1900]"
]

pois_re_model_summary = pois_re_model_stanfit.summary()
pois_re_model_summary.loc[vars]

In [ ]:
mu_index = pois_re_model_summary.index.str.contains("mu")
mu_mean = pois_re_model_summary.loc[mu_index, "Mean"].reset_index(drop=True)
plot_map(nyc_shp_subset.assign(mu = mu_mean), "mu")

#### Adding an ICAR component (spatial smoothing only)

### Visualizing the fitted BYM2 model for New York City and Brooklyn

In [ ]:
mu_index = bym2_nyc_model_summary.index.str.contains("mu")
mu_mean = bym2_nyc_model_summary.loc[mu_index, "Mean"].reset_index(drop=True)
plot_map(nyc_shp_subset.assign(mu = mu_mean), "mu")